<a href="https://colab.research.google.com/github/ChelseaOwiti/Colposcopy-Scan/blob/main/draft2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Project
%ls

/content/drive/MyDrive/Project
d2_model.h5  d3_model.h5  data/  draft1.ipynb  h5_model.h5  model.tflite


In [ ]:
trainpath = 'data/train'
valpath = 'data/val'
testpath = 'data/test'

In [ ]:
import numpy as np
import os
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

import pathlib

In [ ]:
train_dir = pathlib.Path(trainpath)
test_dir = pathlib.Path(valpath)

print(str(len(list(train_dir.glob('*/*.jpg'))) )+ ' Training Images')
print(str(len(list(test_dir.glob('*/*.jpg')))) + ' Testing Images')

888 Training Images
296 Testing Images


In [ ]:
batch_size = 32  
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split = 0.1,
    subset ='training',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 888 files belonging to 3 classes.
Using 800 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split = 0.1,
    subset ='validation',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 888 files belonging to 3 classes.
Using 88 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Type_1', 'Type_2', 'Type_3']


In [ ]:
for image_batch, labels_batch in train_ds: #image batch is a tensor of the shape
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


Transfer learning & fine-tuning with a custom training loop

In [ ]:
# Create base model
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)
# Freeze base model
base_model.trainable = False

# Create new model on top.
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()

# Iterate over the batches of a dataset.
for inputs, targets in train_ds:
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        # Forward pass.
        predictions = model(inputs)
        # Compute the loss value for this batch.
        loss_value = loss_fn(targets, predictions)

    # Get gradients of loss wrt the *trainable* weights.
    gradients = tape.gradient(loss_value, model.trainable_weights)
    # Update the weights of the model.
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))


83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 2,049
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
#train the top layer
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=val_ds)


Epoch 1/20
25/25 [==============================] - 186s 7s/step - loss: -35.5148 - binary_accuracy: 0.5225 - val_loss: -20.9393 - val_binary_accuracy: 0.5682
Epoch 2/20
25/25 [==============================] - 171s 7s/step - loss: -58.3672 - binary_accuracy: 0.5225 - val_loss: -29.7525 - val_binary_accuracy: 0.5682
Epoch 3/20
25/25 [==============================] - 170s 7s/step - loss: -79.8750 - binary_accuracy: 0.5225 - val_loss: -38.2977 - val_binary_accuracy: 0.5682
Epoch 4/20
25/25 [==============================] - 171s 7s/step - loss: -101.2328 - binary_accuracy: 0.5225 - val_loss: -46.9904 - val_binary_accuracy: 0.5682
Epoch 5/20
25/25 [==============================] - 169s 7s/step - loss: -121.6502 - binary_accuracy: 0.5225 - val_loss: -56.9061 - val_binary_accuracy: 0.5682
Epoch 6/20
25/25 [==============================] - 168s 7s/step - loss: -143.6379 - binary_accuracy: 0.5225 - val_loss: -66.0414 - val_binary_accuracy: 0.5682
Epoch 7/20
25/25 [=========================

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
_________________________________________________________________
Epoch 1/10
25/25 [==============================] - 655s 25s/step - loss: -3430.2334 - binary_accuracy: 0.5225

In [ ]:
model.save("d3_model.h5")